In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = irgan_config.epochs_d
epochs_g = irgan_config.epochs_g
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset(irgan_config.dir_path)
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [4]:
dp = DataProvider(device)
evaluator = RecEvaluator(eval_dict, None, device)

In [5]:
writer_dis = SummaryWriter("runs/IRGAN-SGD-dns-gen-dis/Discriminator")
writer_gen = SummaryWriter("runs/IRGAN-SGD-dns-gen-dis/Generator")

In [6]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.SGD(G.parameters(), lr = eta_G)
optimizer_D = torch.optim.SGD(D.parameters(), lr = eta_D, weight_decay = weight_decay_d)

In [7]:
pretrained_model = torch.load("./pretrained_models/pretrained_model_dns.pkl",map_location=irgan_config.device)
G.load_state_dict(pretrained_model)
D.load_state_dict(pretrained_model)

<All keys matched successfully>

In [8]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
        writer_dis.add_scalar('Loss/Discriminator', loss_d_epoch, epoch*epochs_d + epoch_d)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(D, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_dis.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MAP', map10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MRR', mrr10, epoch*epochs_d + epoch_d)
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
        writer_gen.add_scalar('Loss/Generator', loss_g_epoch, epoch*epochs_g + epoch_g)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(G, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_gen.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MAP', map10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MRR', mrr10, epoch*epochs_g + epoch_g)
    time_epoch_end = time.time()
writer_dis.close()    
writer_gen.close()

[Epoch 1/200]
	[Discriminator][Epochs 1/100] [D epoch loss: 3.62872] [Time:7.39089] 
	[Discriminator][Epochs 2/100] [D epoch loss: 3.36076] [Time:8.31594] 
	[Discriminator][Epochs 3/100] [D epoch loss: 3.10142] [Time:8.36656] 
	[Discriminator][Epochs 4/100] [D epoch loss: 2.85400] [Time:8.46270] 
	[Discriminator][Epochs 5/100] [D epoch loss: 2.62253] [Time:8.52264] 
	[Discriminator][Epochs 6/100] [D epoch loss: 2.41131] [Time:9.19447] 
	[Discriminator][Epochs 7/100] [D epoch loss: 2.22207] [Time:8.94545] 
	[Discriminator][Epochs 8/100] [D epoch loss: 2.05535] [Time:8.44775] 
	[Discriminator][Epochs 9/100] [D epoch loss: 1.90964] [Time:9.51699] 
	[Discriminator][Epochs 10/100] [D epoch loss: 1.78252] [Time:9.60930] 
	[Discriminator][Epochs 11/100] [D epoch loss: 1.67166] [Time:10.27783] 
	[Discriminator][Epochs 12/100] [D epoch loss: 1.57440] [Time:10.05733] 
	[Discriminator][Epochs 13/100] [D epoch loss: 1.48901] [Time:9.88129] 
	[Discriminator][Epochs 14/100] [D epoch loss: 1.41386] [

	[Generator][Epochs 16/50] [G epoch loss: -7.78265] [Time:0.03825]
	[Generator][Epochs 17/50] [G epoch loss: -7.72096] [Time:0.03617]
	[Generator][Epochs 18/50] [G epoch loss: -7.79368] [Time:0.04049]
	[Generator][Epochs 19/50] [G epoch loss: -7.70033] [Time:0.04602]
	[Generator][Epochs 20/50] [G epoch loss: -7.99326] [Time:0.04282]
	[Generator][Epochs 21/50] [G epoch loss: -7.83575] [Time:0.04518]
	[Generator][Epochs 22/50] [G epoch loss: -8.09101] [Time:0.05515]
	[Generator][Epochs 23/50] [G epoch loss: -7.80629] [Time:0.03991]
	[Generator][Epochs 24/50] [G epoch loss: -7.78130] [Time:0.03899]
	[Generator][Epochs 25/50] [G epoch loss: -7.86165] [Time:0.02977]
	[Generator][Epochs 26/50] [G epoch loss: -7.45523] [Time:0.03574]
	[Generator][Epochs 27/50] [G epoch loss: -7.25119] [Time:0.03302]
	[Generator][Epochs 28/50] [G epoch loss: -7.46189] [Time:0.02760]
	[Generator][Epochs 29/50] [G epoch loss: -7.01792] [Time:0.02691]
	[Generator][Epochs 30/50] [G epoch loss: -7.16933] [Time:0.02

	[Discriminator][Epochs 82/100] [D epoch loss: 0.42569] [Time:10.40067] 
	[Discriminator][Epochs 83/100] [D epoch loss: 0.42464] [Time:10.11514] 
	[Discriminator][Epochs 84/100] [D epoch loss: 0.42360] [Time:9.06303] 
	[Discriminator][Epochs 85/100] [D epoch loss: 0.42257] [Time:9.28563] 
	[Discriminator][Epochs 86/100] [D epoch loss: 0.42192] [Time:9.74753] 
	[Discriminator][Epochs 87/100] [D epoch loss: 0.42092] [Time:8.33666] 
	[Discriminator][Epochs 88/100] [D epoch loss: 0.41994] [Time:8.74302] 
	[Discriminator][Epochs 89/100] [D epoch loss: 0.41896] [Time:10.26272] 
	[Discriminator][Epochs 90/100] [D epoch loss: 0.41800] [Time:10.22219] 
	[Discriminator][Epochs 91/100] [D epoch loss: 0.41669] [Time:8.51303] 
	[Discriminator][Epochs 92/100] [D epoch loss: 0.41576] [Time:8.89555] 
	[Discriminator][Epochs 93/100] [D epoch loss: 0.41484] [Time:9.56882] 
	[Discriminator][Epochs 94/100] [D epoch loss: 0.41393] [Time:10.47562] 
	[Discriminator][Epochs 95/100] [D epoch loss: 0.41303] [Ti

	[Discriminator][Epochs 49/100] [D epoch loss: 0.44282] [Time:10.32395] 
	[Discriminator][Epochs 50/100] [D epoch loss: 0.44130] [Time:10.08294] 
	[Discriminator][Epochs 51/100] [D epoch loss: 0.44013] [Time:11.08586] 
	[Discriminator][Epochs 52/100] [D epoch loss: 0.43867] [Time:11.23029] 
	[Discriminator][Epochs 53/100] [D epoch loss: 0.43723] [Time:9.62769] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.43581] [Time:10.35661] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.43443] [Time:11.44117] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.43308] [Time:8.92267] 
	[Discriminator][Epochs 57/100] [D epoch loss: 0.43174] [Time:8.66257] 
	[Discriminator][Epochs 58/100] [D epoch loss: 0.43043] [Time:9.82109] 
	[Discriminator][Epochs 59/100] [D epoch loss: 0.42914] [Time:8.88027] 
	[Discriminator][Epochs 60/100] [D epoch loss: 0.42788] [Time:9.27588] 
	[Discriminator][Epochs 61/100] [D epoch loss: 0.42666] [Time:9.42851] 
	[Discriminator][Epochs 62/100] [D epoch loss: 0.42543] [T

	[Discriminator][Epochs 16/100] [D epoch loss: 0.43868] [Time:10.20505] 
	[Discriminator][Epochs 17/100] [D epoch loss: 0.43741] [Time:10.30927] 
	[Discriminator][Epochs 18/100] [D epoch loss: 0.43616] [Time:10.56921] 
	[Discriminator][Epochs 19/100] [D epoch loss: 0.43492] [Time:10.54186] 
	[Discriminator][Epochs 20/100] [D epoch loss: 0.43371] [Time:8.81398] 
	[Discriminator][Epochs 21/100] [D epoch loss: 0.43260] [Time:8.78841] 
	[Discriminator][Epochs 22/100] [D epoch loss: 0.43143] [Time:9.29167] 
	[Discriminator][Epochs 23/100] [D epoch loss: 0.43028] [Time:10.14555] 
	[Discriminator][Epochs 24/100] [D epoch loss: 0.42915] [Time:10.19207] 
	[Discriminator][Epochs 25/100] [D epoch loss: 0.42803] [Time:10.15446] 
	[Discriminator][Epochs 26/100] [D epoch loss: 0.42702] [Time:10.27733] 
	[Discriminator][Epochs 27/100] [D epoch loss: 0.42594] [Time:9.92792] 
	[Discriminator][Epochs 28/100] [D epoch loss: 0.42488] [Time:9.21906] 
	[Discriminator][Epochs 29/100] [D epoch loss: 0.42383] 

...

	[Generator][Epochs 19/50] [G epoch loss: -6.81912] [Time:0.03513]
	[Generator][Epochs 20/50] [G epoch loss: -6.82134] [Time:0.03685]
	[Generator][Epochs 21/50] [G epoch loss: -6.80269] [Time:0.03593]
	[Generator][Epochs 22/50] [G epoch loss: -6.65638] [Time:0.03667]
	[Generator][Epochs 23/50] [G epoch loss: -6.79676] [Time:0.03648]
	[Generator][Epochs 24/50] [G epoch loss: -6.83050] [Time:0.03700]
	[Generator][Epochs 25/50] [G epoch loss: -6.94416] [Time:0.03571]
	[Generator][Epochs 26/50] [G epoch loss: -7.17057] [Time:0.03645]
	[Generator][Epochs 27/50] [G epoch loss: -7.22625] [Time:0.03679]
	[Generator][Epochs 28/50] [G epoch loss: -7.22227] [Time:0.03559]
	[Generator][Epochs 29/50] [G epoch loss: -7.10967] [Time:0.03694]
	[Generator][Epochs 30/50] [G epoch loss: -6.89299] [Time:0.03599]
	[Generator][Epochs 31/50] [G epoch loss: -6.84234] [Time:0.03733]
	[Generator][Epochs 32/50] [G epoch loss: -6.91121] [Time:0.03686]
	[Generator][Epochs 33/50] [G epoch loss: -6.98323] [Time:0.03

	[Discriminator][Epochs 85/100] [D epoch loss: 0.35297] [Time:5.11412] 
	[Discriminator][Epochs 86/100] [D epoch loss: 0.35287] [Time:4.95564] 
	[Discriminator][Epochs 87/100] [D epoch loss: 0.35286] [Time:5.12025] 
	[Discriminator][Epochs 88/100] [D epoch loss: 0.35284] [Time:5.14625] 
	[Discriminator][Epochs 89/100] [D epoch loss: 0.35283] [Time:5.16487] 
	[Discriminator][Epochs 90/100] [D epoch loss: 0.35282] [Time:6.10557] 
	[Discriminator][Epochs 91/100] [D epoch loss: 0.35282] [Time:5.98148] 
	[Discriminator][Epochs 92/100] [D epoch loss: 0.35281] [Time:6.03187] 
	[Discriminator][Epochs 93/100] [D epoch loss: 0.35279] [Time:5.95435] 
	[Discriminator][Epochs 94/100] [D epoch loss: 0.35278] [Time:5.95937] 
	[Discriminator][Epochs 95/100] [D epoch loss: 0.35277] [Time:5.98108] 
	[Discriminator][Epochs 96/100] [D epoch loss: 0.35264] [Time:5.13026] 
	[Discriminator][Epochs 97/100] [D epoch loss: 0.35263] [Time:5.09922] 
	[Discriminator][Epochs 98/100] [D epoch loss: 0.35261] [Time:5.

	[Discriminator][Epochs 53/100] [D epoch loss: 0.35366] [Time:5.12990] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.35364] [Time:5.14444] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.35363] [Time:5.10694] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.35375] [Time:5.16887] 
	[Discriminator][Epochs 57/100] [D epoch loss: 0.35373] [Time:5.15615] 
	[Discriminator][Epochs 58/100] [D epoch loss: 0.35372] [Time:5.09472] 
	[Discriminator][Epochs 59/100] [D epoch loss: 0.35370] [Time:5.21776] 
	[Discriminator][Epochs 60/100] [D epoch loss: 0.35369] [Time:5.25406] 
	[Discriminator][Epochs 61/100] [D epoch loss: 0.35364] [Time:5.11367] 
	[Discriminator][Epochs 62/100] [D epoch loss: 0.35362] [Time:5.11879] 
	[Discriminator][Epochs 63/100] [D epoch loss: 0.35361] [Time:5.14524] 
	[Discriminator][Epochs 64/100] [D epoch loss: 0.35360] [Time:4.87343] 
	[Discriminator][Epochs 65/100] [D epoch loss: 0.35358] [Time:4.84787] 
	[Discriminator][Epochs 66/100] [D epoch loss: 0.35352] [Time:5.

	[Discriminator][Epochs 21/100] [D epoch loss: 0.35515] [Time:5.13821] 
	[Discriminator][Epochs 22/100] [D epoch loss: 0.35513] [Time:5.15505] 
	[Discriminator][Epochs 23/100] [D epoch loss: 0.35512] [Time:5.09591] 
	[Discriminator][Epochs 24/100] [D epoch loss: 0.35511] [Time:5.10090] 
	[Discriminator][Epochs 25/100] [D epoch loss: 0.35509] [Time:5.02008] 
	[Discriminator][Epochs 26/100] [D epoch loss: 0.35481] [Time:4.98752] 
	[Discriminator][Epochs 27/100] [D epoch loss: 0.35480] [Time:5.01927] 
	[Discriminator][Epochs 28/100] [D epoch loss: 0.35478] [Time:5.01662] 
	[Discriminator][Epochs 29/100] [D epoch loss: 0.35477] [Time:5.00106] 
	[Discriminator][Epochs 30/100] [D epoch loss: 0.35476] [Time:5.00323] 
	[Discriminator][Epochs 31/100] [D epoch loss: 0.35457] [Time:4.94137] 
	[Discriminator][Epochs 32/100] [D epoch loss: 0.35455] [Time:4.78388] 
	[Discriminator][Epochs 33/100] [D epoch loss: 0.35454] [Time:4.81276] 
	[Discriminator][Epochs 34/100] [D epoch loss: 0.35453] [Time:5.

	[Generator][Epochs 38/50] [G epoch loss: -7.37504] [Time:0.03543]
	[Generator][Epochs 39/50] [G epoch loss: -7.37662] [Time:0.03297]
	[Generator][Epochs 40/50] [G epoch loss: -7.09695] [Time:0.03921]
	[Generator][Epochs 41/50] [G epoch loss: -7.37989] [Time:0.03428]
	[Generator][Epochs 42/50] [G epoch loss: -7.41963] [Time:0.03829]
	[Generator][Epochs 43/50] [G epoch loss: -7.46049] [Time:0.03639]
	[Generator][Epochs 44/50] [G epoch loss: -7.64568] [Time:0.03360]
	[Generator][Epochs 45/50] [G epoch loss: -7.15170] [Time:0.03566]
	[Generator][Epochs 46/50] [G epoch loss: -7.25439] [Time:0.03534]
	[Generator][Epochs 47/50] [G epoch loss: -7.36852] [Time:0.03546]
	[Generator][Epochs 48/50] [G epoch loss: -7.28588] [Time:0.03552]
	[Generator][Epochs 49/50] [G epoch loss: -7.47148] [Time:0.03622]
	[Generator][Epochs 50/50] [G epoch loss: -7.34835] [Time:0.04011]
